# Stock Price Predictor 

## Pre-requisites

- You will have to pocess the stock's data in json format in a folder location which istrictly follows the following pattern 
    <b>'data/***stockname***/raw/***type-of-data***'</b>
    
    For example, <b>'data/itc/raw/1h' </b>
    
- The folder location that you will mention below should have json files with data associated with only one stock information. If not the please clean it up and until then do not proceed further with this procedure

- You Will have to ensure that the data provided via the json files are clean 
    - should not have duplicates 
    - should not have incomplete, empty or semi furnished information
    - should not have wrong or assumed data which might screw the predictor. 
    
- You will have to configure the following inputs to kick start the prediction process

- <b><u>CAUTION</u></b> : The procedure will create a folder named "processed" parallel to the folder named"raw" as mentioned in the input format. If you have such a folder already in place <b><u> BE WARNED - IT WILL BE OVER WRITTEN </u></b>. You have been warned.

The following procedure does not have any explict check or validation with respect to the items mentioned above


## Configure inputs

In [1]:
raw_data_folder_path = 'data/adani/raw/1h' 
print('done')

done


## Kick start the procedure - Collate all json data into a CSV file

Process all json data files in folder mentioned above and and create a cummulative data file in csv format with desirable column headers to help differnentiate or understand the data during further processing in the folder named "processed"

In [2]:
# Include python files defined in the library folder to help facilitate the stock price Prediction 
import os
import sys  
module_path = os.path.abspath(os.path.join('.'))
sys.path.insert(0, module_path+'\\lib')

print('done')

done


In [3]:
from dataPreparation import processRawData
rawProcessedInputDataInfo=processRawData(raw_data_folder_path)
print('\nprocessed raw data and created processable csv file for data preparation at >>>' + rawProcessedInputDataInfo[1])
print('done')

looping through all the files to create input data
Attempting to create folder if it does not exist >>>C:\Users\kalagi01\Desktop\dev\erai\data\adani\processed
Attempting to create/update file >>>C:\Users\kalagi01\Desktop\dev\erai\data\adani\processed/processedRawData.csv
created raw easy to use csv data to be used for preparing training data in the location  >>>C:\Users\kalagi01\Desktop\dev\erai\data\adani\processed/processedRawData.csv

processed raw data and created processable csv file for data preparation at >>>C:\Users\kalagi01\Desktop\dev\erai\data\adani\processed/processedRawData.csv
done


## Prepare Training Data Set 

In [ ]:
from dataPreparation import createInputData

#rawDataFilePath=rawProcessedInputDataInfo[1]
rawDataFilePath="C:\\Users\\kalagi01\\Desktop\\dev\\erai\\data\\adani\\processed\\processedRawData.csv"
input_data_preparation_response = createInputData(rawDataFilePath)
preparedTrainingDataDF=input_data_preparation_response[3]
print('done')
preparedTrainingDataDF.tail()


In [ ]:
numberOfInputFeatures = preparedTrainingDataDF.shape[1]
numberOfOutputFeatures = 5

numberOfInputFeatures,numberOfOutputFeatures

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(16,8))
plt.plot(preparedTrainingDataDF["open"])
plt.plot(preparedTrainingDataDF["high"])
plt.plot(preparedTrainingDataDF["low"])
plt.plot(preparedTrainingDataDF["close"])
plt.title('Stock price history')
plt.ylabel('Price (Rupees)')
plt.xlabel('Days')
plt.legend(['open','high','low','close'], loc='upper left')
plt.show()
print('done')

In [ ]:
corr = preparedTrainingDataDF.corr()
print('done')
corr

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())



fig, ax = plt.subplots(figsize=(15,15))
sns.heatmap(corr,   xticklabels=corr.columns,     yticklabels=corr.columns)
print('done')

In [ ]:
trainingDataFilePath = input_data_preparation_response[1]
trainingDataFilePath

In [ ]:
from forecast import *
success, trainingData, min_max_scaler, x_t, y_t,x_test_t, y_test_t, lstm_model, history = trainLSTMModel(trainingDataFilePath, TIME_STEPS_CONFIG=60, BATCH_SIZE_CONFIG = 512, LEARNING_RATE_CONFIG=0.001, EPOCHS_CONFIG = 20)
print('done')

# Save trained model for future use

In [ ]:
import pickle

modelFolderPath=getParentFolder(trainingDataFilePath)+"\model"
modelFilePath = modelFolderPath+"\lstm_model.sav"
createFolder(modelFolderPath)

pickle.dump(lstm_model, open(modelFilePath, 'wb'))
print('done')

# Load Model from disk

In [ ]:
import pickle
lstm_model_loaded = pickle.load(open(modelFilePath, 'rb'))
result = lstm_model_loaded.evaluate(x_t, y_t, batch_size=512)
actual = lstm_model.evaluate(x_t, y_t, batch_size=512)
print('done')
actual, result

In [ ]:
result = lstm_model_loaded.evaluate(x_test_t, y_test_t, batch_size=512)
actual = lstm_model.evaluate(x_test_t, y_test_t, batch_size=512)
print('done')
actual, result

In [ ]:
x_t.shape,x_test_t.shape

In [ ]:
predicted_values = lstm_model_loaded.predict(x_test_t, batch_size=512)
predicted_values.shape


In [ ]:
dummy_cols_data = np.random.random((predicted_values.shape[0],numberOfInputFeatures-numberOfOutputFeatures))
predicted_values_adj = np.append(predicted_values, dummy_cols_data, 1)
predicted_values_orig_scale_withDummies=min_max_scaler.inverse_transform(predicted_values_adj)

y_test_t_adj = np.append(y_test_t, dummy_cols_data, 1)
y_test_t_orig_scale_withDummies=min_max_scaler.inverse_transform(y_test_t_adj)
print('done')

In [ ]:
predicted_values_orig_scale= predicted_values_orig_scale_withDummies[:, [1,2,3,4,5]]
y_test_t_orig_scale= y_test_t_orig_scale_withDummies[:, [1,2,3,4,5]]
predicted_values_orig_scale.shape


In [ ]:
plt.figure(figsize=(16,8))
plt.clf()
predicted_open = predicted_values_orig_scale[:,[0]]
actual_open = y_test_t_orig_scale[:,[0]]

x_coord = [ index for index in range(predicted_values_orig_scale.shape[0]) ]

plt.plot(x_coord, predicted_open, 'g', label='Predicted Open')
plt.plot(x_coord, actual_open, 'y', label='Actual Open')
plt.title('Open -> Predicted vs Actual')
plt.xlabel('Range')
plt.ylabel('Open Price')
plt.legend()
plt.show()

In [ ]:
predicted_values_orig_scale.shape[0]